In [ ]:
from petastorm import make_reader
import numpy as np
import sys
import os
import pandas as pd
import seaborn as sn
import mogptk as mo
sys.path.append(os.path.join(os.getcwd(), ".."))
path_formatted_glasgow_disease = "/workspaces/maitrise/data/20220902_data_physio_formatted_merged/merged/dataParquet"
path_petastorm_disease = f"file:///{path_formatted_glasgow_disease}"
path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
path_csv_ref_label = "/workspaces/maitrise/data/20221006_physio_quality/set-a/REFERENCE.csv"

In [ ]:
##Dataset Organization : 
with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        if data.signal_quality == "acceptable".encode():
            break
        else : 
            continue

ECG_lead = [e.decode('UTF-8') for e in data.signal_names]
fs = data.sampling_frequency
dico_ECG = {}
for i,j in zip(ECG_lead,range(12)):
    
    dico_ECG[i] = data.signal[:,j]
N = len(dico_ECG[ECG_lead[0]])
print(data)

In [ ]:
##Dataset with MOGPTK
t = np.linspace(0,N/fs,N)

# dataset = mo.DataSet(mo.Data(t,dico_ECG[ECG_lead[0]],name = ECG_lead[0]),
#                      mo.Data(t,dico_ECG[ECG_lead[1]],name = ECG_lead[1]),
#                      mo.Data(t,dico_ECG[ECG_lead[2]],name = ECG_lead[2]),
#                      mo.Data(t,dico_ECG[ECG_lead[3]],name = ECG_lead[3]),
#                      mo.Data(t,dico_ECG[ECG_lead[4]],name = ECG_lead[4]),
#                      mo.Data(t,dico_ECG[ECG_lead[5]],name = ECG_lead[5]),
#                      mo.Data(t,dico_ECG[ECG_lead[6]],name = ECG_lead[6]),
#                      mo.Data(t,dico_ECG[ECG_lead[7]],name = ECG_lead[7]),
#                      mo.Data(t,dico_ECG[ECG_lead[8]],name = ECG_lead[8]),
#                      mo.Data(t,dico_ECG[ECG_lead[9]],name = ECG_lead[9]),
#                      mo.Data(t,dico_ECG[ECG_lead[10]],name = ECG_lead[10]),
#                      mo.Data(t,dico_ECG[ECG_lead[11]],name = ECG_lead[11]))

dataset = mo.DataSet(mo.Data(t,dico_ECG[ECG_lead[0]],name = ECG_lead[0]),
                     mo.Data(t,dico_ECG[ECG_lead[1]],name = ECG_lead[1]),
                     mo.Data(t,dico_ECG[ECG_lead[2]],name = ECG_lead[2]),
                     mo.Data(t,dico_ECG[ECG_lead[3]],name = ECG_lead[3]),
                     mo.Data(t,dico_ECG[ECG_lead[7]],name = ECG_lead[7]))

# df = pd.DataFrame(dico_ECG)
# df["Time"] = t

# sel = list([ECG_lead[i] for i in range(0,3)])
# dataset = mo.LoadDataFrame(df,x_col ="Time",y_col = sel)
# dataset

In [ ]:
##Removing datapoints in dataset :
class TransformStandard(mo.TransformBase):
    """
    Transform the data so it has mean 0 and variance 1
    """
    def __init__(self):
        pass
    
    def set_data(self, y, x=None):
        self.mean = y.mean()
        self.std = y.std()
        
    def forward(self, y, x=None):
        return (y - self.mean) / self.std
    
    def backward(self, y, x=None):
        return (y * self.std) + self.mean

start_ = 1
end_ = 5
for channel in dataset:
    channel.filter(start_, end_)

for i, channel in enumerate(dataset):
    if i == 0:
        channel.remove_range(start_,end_-1)
        channel.remove_randomly(pct=0.4)
    if i == 1:
        #channel.remove_range(1,2)
        channel.remove_randomly(pct=0.4)
    if i == 2:
        #channel.remove_range(2,3)
        channel.remove_randomly(pct=0.4)
    if i == 3:
        #channel.remove_range(1.5,2.5)
        channel.remove_randomly(pct=0.4)
    if i == 4:
        #channel.remove_range(3,4)
        channel.remove_randomly(pct=0.4)
        

dataset.transform(TransformStandard())

dataset.plot('Normalized Dataset')

In [ ]:
method = "Adam"
iters = 100
lr = 0.01
init_par = "BNSE"
error_used = "MAE"
Q = 5

In [ ]:
##Model Spectral Mixture kernels : 
sm = mo.SM(dataset,Q = Q)
sm.init_parameters(init_par)
sm.train(method = method,lr = lr,iters = iters,verbose = True,error = error_used,plot = True)
sm.plot_prediction(title = "SM for ECG reconstruction ")

In [ ]:
# create model
mosm = mo.MOSM(dataset,Q = Q)
mosm.init_parameters(init_par)
mosm.train(method = method,lr = lr,iters = iters,verbose = True,error = error_used,plot = True)
mosm.plot_prediction(title = "MOSM for ECG reconstruction using ")

In [ ]:
csm= mo.CSM(dataset,Q = Q)
csm.init_parameters(init_par)
csm.train(method = method,lr = lr,iters = iters,verbose = True,error = error_used,plot = True)
csm.plot_prediction(title = "CSM for ECG reconstruction using ")

In [ ]:
smlmc= mo.SM_LMC(dataset,Q = Q)
smlmc.init_parameters(init_par)
smlmc.train(method = method,lr = lr,iters = iters,verbose = True,error = error_used,plot = True)
smlmc.plot_prediction(title = "SM_LMC for ECG reconstruction")

In [ ]:
conv= mo.CONV(dataset,Q = Q)
conv.init_parameters(init_par)
conv.train(method = method,lr = lr,iters = iters,verbose = True,error = error_used,plot = True)
conv.plot_prediction(title = "Conv for ECG reconstruction")

In [ ]:
mo.error(sm, mosm, csm, smlmc, conv, disp=True)